In [1]:
import cv2 
import csv
import numpy as np 
from cv2 import aruco
import math
import networkx as nx
import time
import socket

In [2]:
def detect_ArUco_details(image): 
    ArUco_details_dict = {}
    ArUco_corners = {}
    
    ##############	ADD YOUR CODE HERE	##############
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    arucoParams = aruco.DetectorParameters()
    # GrayScale Conversion
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, _ = aruco.detectMarkers(gray_image, aruco_dict, parameters=arucoParams)

    if ids is not None:
        for i in range(len(ids)):
            marker_id = int(ids[i][0])
            marker_center = [int(coord) for coord in list(np.mean(corners[i][0], axis=0).astype(int))]

            # Store details in dictionaries
            ArUco_details_dict[marker_id] = [marker_center, 0]
            ArUco_corners[marker_id] = [[int(corner[0]), int(corner[1])] for corner in corners[i][0]]
    ##################################################
    
    return ArUco_details_dict, ArUco_corners 

In [3]:
def mark_ArUco_image(image,ArUco_details_dict, ArUco_corners):

    for ids, details in ArUco_details_dict.items():
        center = details[0]

        corner = ArUco_corners[int(ids)]

        tl_tr_center_x = int((corner[0][0] + corner[1][0]) / 2)
        tl_tr_center_y = int((corner[0][1] + corner[1][1]) / 2) 

        display_offset = int(math.sqrt((tl_tr_center_x - center[0])**2+(tl_tr_center_y - center[1])**2))
        cv2.putText(image,str(ids),(center[0]+int(display_offset/2),center[1]),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
    return image

In [4]:
def sort_labels(identified_labels, targets):
    order = ['Fire', 'Destroyed buildings', 'Humanitarian Aid and rehabilitation', 'Military Vehicles', 'Combat']
    result = []
    for target in targets:
        inserted = 0
        tpos = order.index(identified_labels[target])
        if len(result) != 0:
            for ind, key in enumerate(result):
                rpos = order.index(identified_labels[key])
                if tpos <= rpos:
                    result.insert(ind, target)
                    inserted = 1
                    break
        if not inserted:
            result.append(target)
    return result

In [5]:
def distance(ar1, ar2):
    c1 = ar1
    x1, y1 = c1[0], c1[1]
    c2 = ar2
    x2, y2 = c2[0], c2[1]

    width = x2-x1
    height = y2-y1
    dist = math.sqrt(pow(width, 2) + pow(height, 2))
    return dist

In [6]:
def rotate_coordinates(x, y, theta_degrees):
    # Convert theta from degrees to radians
    theta = math.radians(-theta_degrees)

    # Perform the rotation
    x_prime = x * math.cos(theta) - y * math.sin(theta)
    y_prime = x * math.sin(theta) + y * math.cos(theta)

    return x_prime, y_prime

In [7]:
def adjust_coordinates(csv_name, theta_degrees):
    adjusted_coordinates = {}

    with open(csv_name, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for row in csv_reader:
            ar_id, lat, lon = row[0], float(row[1]), float(row[2])
            adjusted_lat, adjusted_lon = rotate_coordinates(lat, lon, theta_degrees)
            adjusted_coordinates[ar_id] = [adjusted_lat, adjusted_lon]

    return adjusted_coordinates

In [8]:
def create_graph():
    links = (
        (23, 24), (24, 22), (22, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 48), (48, 47), (47, 46), 
        (46, 45), (45, 44), (44, 43), (43, 10), (10, 8), (8, 12), (12, 9), (9, 11), (11, 13), (13, 14), (14, 15), 
        (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 23),

        (22, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 11),

        (50, 34), (34, 33), (33, 32), (32, 31), (31, 30), (30, 12),

        (51, 42), (42, 41), (41, 40), (40, 39), (39, 35), (35, 38), (38, 37), (37, 36), (36, 10), (36, 8),

        (19, 27), (19, 28), (19, 32),

        (27, 32), (28, 32),

        (32, 39), (32, 35)       
    )
    nodes = [int(coord) for coord in coords.keys()]
    init_graph = {node: {} for node in nodes}
    for n1, n2 in links:
        init_graph[n1][n2] = distance(coords[str(n1)], coords[str(n2)])
    graph = nx.Graph()
    for link in links:
        graph.add_edge(link[0], link[1], weight=init_graph[link[0]][link[1]])
    return graph

In [9]:
def isNode(node, traversed):
    turns = ((23,), (19,), (22,), (27, 28), (11,), (50,), (32,), (12,), (51,), (39, 35), (10, 8))
    for turn in turns:
        if (node in turn) and turns.index(turn) not in traversed:
            traversed.append(turns.index(turn))
            return True, traversed
    return False, traversed

In [10]:
def calculate_angle(coord1, coord2, coord3):
    # Calculate the distances between the points
    a = math.sqrt((coord2[0] - coord1[0])**2 + (coord2[1] - coord1[1])**2)
    b = math.sqrt((coord3[0] - coord2[0])**2 + (coord3[1] - coord2[1])**2)
    c = math.sqrt((coord3[0] - coord1[0])**2 + (coord3[1] - coord1[1])**2)

    # Apply the law of cosines to find the angle
    cos_angle = (a**2 + b**2 - c**2) / (2 * a * b)
    angle = math.acos(cos_angle)

    # Calculate the cross product
    cross_product = (coord2[0] - coord1[0]) * (coord3[1] - coord1[1]) - (coord2[1] - coord1[1]) * (coord3[0] - coord1[0])

    # Determine the direction
    if cross_product > 0:
        direction = 2
    elif cross_product < 0:
        direction = 3
    else:
        direction = "C"

    # Convert the angle to degrees
    angle = math.degrees(angle)

    return angle, direction

In [11]:
def event_angle(coord1, coord2):
    # coord1 and coord2 are tuples representing (x, y)
    x1, y1 = coord1
    x2, y2 = coord2

    # Calculate the difference between the two points
    dx = x2 - x1
    dy = y2 - y1

    # Calculate the angle in radians between the x-axis and a line segment from the origin (0,0) to the point (dx, dy)
    rad = math.atan2(dy, dx)

    # Convert the angle to degrees
    deg = math.degrees(rad)

    cross_product = x1 * y2 - x2 * y1

    side = "left" if cross_product > 0 else "right" if cross_product < 0 else "collinear"

    return deg, side

In [12]:
def path_gen(graph, events):
    event_markers = {
        'A': 21,
        'B': 29,
        'C': 30,
        'D': 34,
        'E': 48
    }
    path = [[] for _ in range(len(events)+1)]
    curr_node = 23
    for ind, event in enumerate(events):
        path[ind].extend(nx.shortest_path(graph, curr_node, event_markers[event], weight='weight'))
        curr_node = event_markers[event]
    path[-1].extend(nx.shortest_path(graph, curr_node, 23, weight='weight'))
    return path

In [13]:
def draw_shortest_path(img, details, path):
    color = (255, 0, 0)
    thickness = 2

    for i in range(len(path) - 1):
        node1 = path[i]
        node2 = path[i + 1]

        x1, y1 = details[node1][0]
        x2, y2 = details[node2][0]

        cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [14]:
def command_gen(graph, coords, events):
    # 1 is for FORWARD till node detection
    # 2 is for RIGHT turn then FORWARD till node detection
    # 3 is for LEFT turn then FORWARD till node detection
    # 4 is for 180 degree turn then FORWARD till node detection
    paths = path_gen(graph, events)
    commands = []
    buffer = 0
    for path in paths:
        c = []
        traversed = []
        if buffer == path[1]:
            c.append(4)
        else:
            c.append(1)
        for i in range(0, len(path)):

            if path[i]==23 and i==0:
                if path[i+1] == 24:
                    c.append(1)
                else:
                    c.append(2)
            elif i<len(path)-2:
                ang, dir = calculate_angle(coords[str(path[i])], coords[str(path[i+1])], coords[str(path[i+2])])
                result, traversed = isNode(path[i+1], traversed)
                if 140 >= ang >= 45:
                    traversed = []
                    c.append(dir)
                    if path[i+2] == 32 and path[i+1] == 19:
                        c.append(1)
                    
                if (170 <= ang <= 180) and result:
                    c.append(1)
            elif path[-1] == 23:
                if path[i] == 21:
                    c.append(3)
                elif path[i] == 24:
                    c.append(1)
        buffer = path[-2]
        commands.append(c)
    return commands
    


In [15]:
# img = cv2.imread('evalpic.jpg')
# details, corners = detect_ArUco_details(img)
# img = mark_ArUco_image(img, details, corners)
# cv2.imshow('Marked', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [16]:
coords = adjust_coordinates('lat_long.csv', -15)

graph = create_graph()


In [17]:
# print(command_gen(graph, coords, ['A', 'B', 'C']))

In [18]:
def read_csv(csv_name):
    lat_lon = {}

    # open csv file (lat_lon.csv)
    # read "lat_lon.csv" file
    # store csv data in lat_lon dictionary as {id:[lat, lon].....}
    # return lat_lon

    with open(csv_name, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            ar_id, lat, lon = row[0], row[1], row[2]
            lat_lon[ar_id] = [lat, lon]

    return lat_lon

def write_csv(loc, csv_name):

    # open csv (csv_name)
    # write column names "lat", "lon"
    # write loc ([lat, lon]) in respective columns

    with open(csv_name, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(["lat", "lon"])  # Write the column names
        for coordinate in loc.values():
            lat, lon = coordinate
            csv_writer.writerow([lat, lon])

def tracker(ar_id, lat_lon):

    # find the lat, lon associated with ar_id (aruco id)
    # write these lat, lon to "live_data.csv"

    coordinate = None

    # Check if the ARUCO ID exists in the lat_lon dictionary
    if str(ar_id) in list(lat_lon.keys()):
        coordinate = lat_lon[str(ar_id)]

        # Write the coordinate to "live_data.csv"
        write_csv({ar_id: coordinate}, "live_data.csv")

    # also return coordinate ([lat, lon]) associated with respective ar_id.
    return coordinate

# lat_lon = read_csv('lat_long.csv')
# tracker(23, lat_lon)

In [ ]:
conversion = {1: 9, 2: 7, 3: 8}

In [19]:
esp32_ip = ""  # Change this to the IP address of your ESP32
esp32_port = 8002

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((esp32_ip, 8002))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            # data = conn.recv(1024)
            # print(data)
            command = input("Enter command (1: Move Forward): ")
            # conn.sendall(str.encode(str(command)))
            # data = conn.recv(1024)
            # priority_list = sort_lables()      ##IDK WHAT ARG
            priority_list = ['D','B']
            command_list = command_gen(graph, coords, priority_list) 
            j = -1
            print("hallo")
            data = "b\'node\'"
            print(command_list)
            print(data)
            for sub in command_list:
                i = 0
                j += 1
                for subsub in sub:
                    i += 1
                    if i == len(sub):
                        conn.sendall(str.encode(str(conversion[subsub])))
                        print(f"command: {subsub}")
                        while data != "turned":
                            data = conn.recv(1024)
                            data = data.decode('utf-8').strip()
                        print(data)
                    else:
                        conn.sendall(str.encode(str(subsub)))
                        print(f"command: {subsub}")
                        while data != "node":
                            data = conn.recv(1024)
                            data = data.decode('utf-8').strip()
                        print(data)
                            command = 4
                            conn.sendall(str.encode(str(command)))
                            print(f"command: {command}")
                            conn.recv(1024)
                    print(f"command processed: {subsub}")
                    data = "out of sub"
                # while i == len(sub):
                #     if atEvent(100, priority_list[j], ----):     #dunno 3rd param    
                #         command = 5                #WAIT and BUZZ
                #         conn.sendall(str.encode(str(command)))
                #         conn.recv(1024)
                #         break
                #     else:
                #         command = 1
                #         conn.sendall(str.encode(str(command)))
                      #   conn.recv(1024)
                # start_time = time.time()
                # while (time.time() - start_time) < 2:
                #     conn.sendall(str.encode(str(1)))
                #     data = conn.recv(1024)
                #     print(data)
                
                command = 5
                conn.sendall(str.encode(str(command)))
                print(f"Sent: {command}")
                data = conn.recv(1024)
                print(data)
                print("done with one event")
            conn.sendall(str.encode(str(1)))
            conn.recv(1024)
            print("helped everyone")
            s.close()



Connected by ('192.168.137.221', 59061)


hallo
[[1, 1, 1, 2], [1, 1], [4, 3, 2, 3, 1]]
b'node'
command: 1
node
command processed: 1
command: 1
node
command processed: 1
command: 1
node
command processed: 1
command: 7
turned
command processed: 7
Sent: 5
b'buzz'
done with one event
command: 1
node
command processed: 1
command processed: 1
Sent: 5
b'buzz'
done with one event
command: 4
